In [1]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import json 
from pandas.io.json import json_normalize 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

!pip install geocoder
print('  >>>>>   Installing Geocoder........ Please wait')

print('   :::::  COMPLETED  :::::  ')

print(' >>>>>>>>>>>>>>>>>>>>>>>>    :::::  Libraries Ready  ::::')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

  >>>>>   Installing Geocoder........ Please wait
   :::::  COMPLETED  :::::  
 >>>>>>>>>>>>>>>>>>>>>>>>    :::::  Libraries Ready  ::::


In [2]:
# acquiring data from link
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page= requests.get(wiki_link).text

soup = BeautifulSoup(page,'xml') # parse the HTML/XML codes using BeautifulSoup.
print(                                                                                  )
print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: ACQUIRED DATA ::::: ')

# extracting the raw data
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'): # cleaning data from the table
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: EXTRACTION DONE ::::: ')

unique_p = set(Postcode)
print('Number of unique Post codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: EXECUTED ::::: ')


>>>>>>>>>>>>>>>>>>>>>>>> ::::: ACQUIRED DATA ::::: 
>>>>>>>>>>>>>>>>>>>>>>>> ::::: EXTRACTION DONE ::::: 
Number of unique Post codes: 77
>>>>>>>>>>>>>>>>>>>>>>>> ::::: EXECUTED ::::: 


In [3]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto1.csv')
df_toronto.shape

(77, 3)

In [4]:
df = pd.read_csv('toronto1.csv') 
print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: LOADED DATASET FOR TORONTO POST CODES ::::: ')

>>>>>>>>>>>>>>>>>>>>>>>> ::::: LOADED DATASET FOR TORONTO POST CODES ::::: 


In [5]:
coordinates_df = pd.read_csv('http://cocl.us/Geospatial_data') 
coordinates_df = df.join(coordinates_df.set_index('Postal Code'), on='Postcode')
coordinates_df.to_csv('coordinates.csv')
coordinates_df.head(20)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,1,M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que...",43.636258,-79.498509
2,2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
3,3,M6K,West Toronto,"Exhibition Place, Parkdale Village",43.636847,-79.428191
4,4,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
5,5,M9L,North York,Humber Summit,43.756303,-79.565963
6,6,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
7,7,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577
8,8,M2R,North York,Willowdale West,43.782736,-79.442259
9,9,M5R,Central Toronto,"The Annex, Yorkville",43.672710,-79.405678


In [6]:
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(coordinates_df['Latitude'], coordinates_df['Longitude'], coordinates_df['Borough'], coordinates_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [7]:
# Area of focus is "Scarborough".
coordinates_df['Borough'] == 'Scarborough'

scarborough_data = coordinates_df[coordinates_df['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True).drop(columns = 'Unnamed: 0')
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577
1,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302
2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
3,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
4,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029


In [8]:
addy_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('Geo-coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

Geo-coordinate of Scarborough are 43.773077, -79.257774.


In [9]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    dx_result = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        x_dict = {}
        x_dict['Postal Code'] = postal_code; x_dict['Neighborhood(s)'] = neighborhood; 
        x_dict['Latitude'] = lat; x_dict['Longitude'] = lng;
        x_dict['Crawling_result'] = results;
        dx_result.append(x_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return dx_result;
print(                                                                                  )
print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: DONE ::::: ')


>>>>>>>>>>>>>>>>>>>>>>>> ::::: DONE ::::: 


In [10]:
CLIENT_ID = 'HNSCZV3FCJQ3D1W4AZHNXBMKQHCH0DKNMVJITR2FMMFA2ACJ' # your Foursquare ID
CLIENT_SECRET = 'HSHSVJ2RQ1Q01CIBPHCJNTMUHW1ZGDIMTVEBL3F01LLKVMKO' # your Foursquare Secret
VERSION = '20200221' # Foursquare API version

print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')


s4s_dataset = foursquare_crawler(list(scarborough_data['Postcode']),
                                                   list(scarborough_data['Neighbourhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Your credentails:
CLIENT_ID: HNSCZV3FCJQ3D1W4AZHNXBMKQHCH0DKNMVJITR2FMMFA2ACJ
CLIENT_SECRET:HSHSVJ2RQ1Q01CIBPHCJNTMUHW1ZGDIMTVEBL3F01LLKVMKO
>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
1.
Data is Obtained, for the Postal Code M1V (and Neighborhoods Agincourt North, Milliken) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1T (and Neighborhoods Tam O'Shanter) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Rouge, Malvern) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1K (and Neighborhoods Ionview, Kennedy Park) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1L (and Neighborhoods Clairlea, Golden Mile, Oakridge) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1S (and Neighborhoods Agincourt) SUCCESSFULLY.
8.
Data is Obtained, for 

### Breakpoint:Saving results of Foursquare, so that we would not need to connect every time to Foursquare (and use our portions) .

In [11]:
import pickle
with open("s4s_dataset.txt", "wb") as og:  
    pickle.dump(s4s_dataset, og)

In [12]:
with open("s4s_dataset.txt", "rb") as og:  
    s4s_dataset = pickle.load(og)

In [13]:
# This function is created to connect to the saved list which is the received database. It will extract each venue 
# for every neighborhood inside the database

def venue_dataset(s4s_dataset):
    result_1 = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
         # print(result_df)
        
    for n_dict in s4s_dataset:
        postal_code = n_dict['Postal Code']; neigh = n_dict['Neighborhood(s)']
        lat = n_dict['Latitude']; lng = n_dict['Longitude']
        print('Number of Venues "{}" Postal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(n_dict['Crawling_result']))
        
        for v_dict in n_dict['Crawling_result']:
            summary = v_dict['reasons']['items'][0]['summary']
            name = v_dict['venue']['name']
            dist = v_dict['venue']['location']['distance']
            cat =  v_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_1 = result_1.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_1)
print(                                                                                  )
print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: DONE ::::: ')


>>>>>>>>>>>>>>>>>>>>>>>> ::::: DONE ::::: 


In [14]:
scarb_venues = venue_dataset(s4s_dataset)

Number of Venues "M1V" Postal Code and "Agincourt North, Milliken" Negihborhood(s) is:
27
Number of Venues "M1T" Postal Code and "Tam O'Shanter" Negihborhood(s) is:
36
Number of Venues "M1J" Postal Code and "Scarborough Village" Negihborhood(s) is:
12
Number of Venues "M1B" Postal Code and "Rouge, Malvern" Negihborhood(s) is:
18
Number of Venues "M1K" Postal Code and "Ionview, Kennedy Park" Negihborhood(s) is:
28
Number of Venues "M1L" Postal Code and "Clairlea, Golden Mile, Oakridge" Negihborhood(s) is:
30
Number of Venues "M1S" Postal Code and "Agincourt" Negihborhood(s) is:
44
Number of Venues "M1N" Postal Code and "Birch Cliff" Negihborhood(s) is:
13
Number of Venues "M1X" Postal Code and "Upper Rouge" Negihborhood(s) is:
0
Number of Venues "M1M" Postal Code and "Cliffcrest, Cliffside" Negihborhood(s) is:
11
Number of Venues "M1C" Postal Code and "Highland Creek, Rouge Hill, Port Union" Negihborhood(s) is:
5
Number of Venues "M1P" Postal Code and "Dorset Park, Scarborough Town Cent

In [15]:
from tabulate import tabulate
data = [[22, 'Morningside, West Hill', 'M1E'], 
        [10, 'Woburn', 'M1G'],
        [13, 'Birch Cliff', 'M1N'], 
        [18, 'Rouge, Malvern', 'M1B'],
        [36, 'Tam O.Shanter', 'M1T'],
        [11, 'Cliffcrest, Cliffside', 'M1M'],
        [44, 'Agincourt', 'M1S'], 
        [5, 'Highland Creek, Rouge Hill, Port Union', 'M1C'],
        [27, 'Agincourt North, Milliken', 'M1V'],
        [28, 'Ionview, Kennedy Park', 'M1K'],
        [30, 'Maryvale, Wexford', 'M1R'],
        [47, 'Dorset Park', 'M1P'],
        [12, 'Scarborough Village', 'M1J'],
        [0, 'Upper Rouge', 'M1X'],
        [30, 'Clairlea, Golden Mile, Oakridge', 'M1L']]

print (tabulate(data, headers=["Number of Venues", "Neighbourhood", "Post Code"]))


  Number of Venues  Neighbourhood                           Post Code
------------------  --------------------------------------  -----------
                22  Morningside, West Hill                  M1E
                10  Woburn                                  M1G
                13  Birch Cliff                             M1N
                18  Rouge, Malvern                          M1B
                36  Tam O.Shanter                           M1T
                11  Cliffcrest, Cliffside                   M1M
                44  Agincourt                               M1S
                 5  Highland Creek, Rouge Hill, Port Union  M1C
                27  Agincourt North, Milliken               M1V
                28  Ionview, Kennedy Park                   M1K
                30  Maryvale, Wexford                       M1R
                47  Dorset Park                             M1P
                12  Scarborough Village                     M1J
                 0  Upper 

### Showing Venues for Each Neighborhood in Scarborough

In [16]:
scarb_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Fahmee Bakery & Jamaican Foods,This spot is popular,Caribbean Restaurant,669
1,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Jim Chai Kee Wonton Noodle 沾仔記,This spot is popular,Noodle House,689
2,M1V,"Agincourt North, Milliken",43.815252,-79.284577,DaanGo Cake Lab,This spot is popular,Bakery,809
3,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Lotus Pond Vegetarian Restaurant 蓮花素食,This spot is popular,Vegetarian / Vegan Restaurant,934
4,M1V,"Agincourt North, Milliken",43.815252,-79.284577,The Brighton Convention & Event Centre,This spot is popular,Event Space,890


In [17]:
scarb_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
328,M1E,"Morningside, West Hill",43.763573,-79.188711,Food Basics,This spot is popular,Supermarket,760
329,M1E,"Morningside, West Hill",43.763573,-79.188711,Pizza Pizza,This spot is popular,Pizza Place,542
330,M1E,"Morningside, West Hill",43.763573,-79.188711,No Frills,This spot is popular,Grocery Store,675
331,M1E,"Morningside, West Hill",43.763573,-79.188711,Lick's Homeburgers,This spot is popular,Burger Joint,941
332,M1E,"Morningside, West Hill",43.763573,-79.188711,Tim Hortons,This spot is popular,Coffee Shop,708


### Breakpoint:End of Processing the Retrieved Information from Foursquare
Saving a Cleaned Version of DataFrame as the Results from Foursquare

In [18]:
scarb_venues.to_csv('venues.csv')
scarb_venues = pd.read_csv('venues.csv')
print(                                                                                  )
print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: DONE ::::: ')


>>>>>>>>>>>>>>>>>>>>>>>> ::::: DONE ::::: 


### Some Summary Information about Neighborhoods inside "Scarborough"

In [19]:
hood_list = list(scarb_venues['Neighborhood'].unique())
print('# of Neighborhoods:')
print(len(hood_list))
print('List of Neighborhoods:')
hood_list

# of Neighborhoods:
14
List of Neighborhoods:


['Agincourt North, Milliken',
 "Tam O'Shanter",
 'Scarborough Village',
 'Rouge, Malvern',
 'Ionview, Kennedy Park',
 'Clairlea, Golden Mile, Oakridge',
 'Agincourt',
 'Birch Cliff',
 'Cliffcrest, Cliffside',
 'Highland Creek, Rouge Hill, Port Union',
 'Dorset Park, Scarborough Town Centre, Wexford Heights',
 'Woburn',
 'Maryvale, Wexford',
 'Morningside, West Hill']

In [33]:
summary = scarb_venues.groupby('Neighborhood').count()
summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,44,44,44,44,44,44,44
"Agincourt North, Milliken",27,27,27,27,27,27,27
Birch Cliff,13,13,13,13,13,13,13
"Clairlea, Golden Mile, Oakridge",30,30,30,30,30,30,30
"Cliffcrest, Cliffside",11,11,11,11,11,11,11


In [21]:
print(' {} uniques categories.'.format(len(scarb_venues['Venue Category'].unique())))
print('List of different Categories:')
list(scarb_venues['Venue Category'].unique())

 104 uniques categories.
List of different Categories:


['Caribbean Restaurant',
 'Noodle House',
 'Bakery',
 'Vegetarian / Vegan Restaurant',
 'Event Space',
 'Malay Restaurant',
 'Dessert Shop',
 'Chinese Restaurant',
 'Coffee Shop',
 'Korean Restaurant',
 'Pizza Place',
 'Hobby Shop',
 'Park',
 'Fast Food Restaurant',
 'Gym',
 'Pharmacy',
 'Shop & Service',
 'Shopping Mall',
 'Grocery Store',
 'Italian Restaurant',
 'Seafood Restaurant',
 'Thai Restaurant',
 'Mexican Restaurant',
 'Greek Restaurant',
 'Bank',
 'Cantonese Restaurant',
 'Sandwich Place',
 'Gas Station',
 'Fried Chicken Joint',
 'Vietnamese Restaurant',
 'Convenience Store',
 'Bus Stop',
 'Intersection',
 'Golf Course',
 'Taiwanese Restaurant',
 'Deli / Bodega',
 'Ice Cream Shop',
 'Restaurant',
 'Train Station',
 'Japanese Restaurant',
 'Bowling Alley',
 'Paper / Office Supplies Store',
 'Spa',
 'Bus Station',
 'Filipino Restaurant',
 'Trail',
 'Fruit & Vegetable Store',
 'Arts & Crafts Store',
 'Department Store',
 'Discount Store',
 'Burger Joint',
 'Metro Station',
 'Li

In [22]:
# one hot encoding
sconehot = pd.get_dummies(data = scarb_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
sconehot.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,American Restaurant,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hardware Store,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Spa,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Fahmee Bakery & Jamaican Foods,This spot is popular,669,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Jim Chai Kee Wonton Noodle 沾仔記,This spot is popular,689,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1V,"Agincourt North, Milliken",43.815252,-79.284577,DaanGo Cake Lab,This spot is popular,809,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Lotus Pond Vegetarian Restaurant 蓮花素食,This spot is popular,934,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,M1V,"Agincourt North, Milliken",43.815252,-79.284577,The Brighton Convention & Event Centre,This spot is popular,890,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 
Manually Selecting (Subsetting) Related Features for the Groceries Contractor

In [23]:
# This was created manually 
sig = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'American Restaurant',
 'Asian Restaurant',
 'Bakery',
 'Breakfast Spot',
 'Burger Joint',
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Diner',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 'Fruit & Vegetable Store',
 'Greek Restaurant',
 'Grocery Store',
 'Hong Kong Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Noodle House',
 'Pizza Place',
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shanghai Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant',
 'Wings Joint']

print(                                                                                  )
print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: DONE ::::: ')


>>>>>>>>>>>>>>>>>>>>>>>> ::::: DONE ::::: 


### Grouping the Data by Neighborhoods

In [24]:
sconehot = sconehot[sig].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()
sconehot.head()

,American Restaurant,Asian Restaurant,Bakery,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Greek Restaurant,Grocery Store,Hong Kong Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,1,1,2,1,0,0,1,2,6,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,1,1,2,1,1,1,0,0,0,1,0
"Agincourt North, Milliken",0,0,2,0,0,0,0,1,5,0,2,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,2,0,0,0,0,0,0,0,1,0,0
Birch Cliff,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
"Clairlea, Golden Mile, Oakridge",0,0,2,0,0,0,0,0,0,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0
"Cliffcrest, Cliffside",0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0


### 
Integrating Different Restaurants and Different Joints

In [25]:
peat = list(sconehot.columns)
restaurant_list = []


for counter, value in enumerate(peat):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
sconehot['Total Restaurants'] = sconehot[restaurant_list].sum(axis = 1)
sconehot = sconehot.drop(columns = restaurant_list)


peat = list(sconehot.columns)
joint_list = []


for counter, value in enumerate(peat):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
sconehot['Total Joints'] = sconehot[joint_list].sum(axis = 1)
sconehot = sconehot.drop(columns = joint_list)

sconehot # Fully-Processed Data


,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,,
Agincourt,2,1,0,0,0,0,0,1,1,2,22,0
"Agincourt North, Milliken",2,0,0,0,0,0,1,1,2,0,11,0
Birch Cliff,0,0,1,0,0,0,0,0,0,0,2,0
"Clairlea, Golden Mile, Oakridge",2,0,2,0,0,0,1,0,1,1,3,0
"Cliffcrest, Cliffside",0,0,0,0,0,0,0,0,2,0,1,1
"Dorset Park, Scarborough Town Centre, Wexford Heights",3,0,0,0,0,0,0,0,1,1,14,3
"Highland Creek, Rouge Hill, Port Union",0,1,0,0,0,0,0,0,0,0,1,1
"Ionview, Kennedy Park",0,0,0,0,0,0,2,0,1,1,6,1
"Maryvale, Wexford",1,1,0,1,0,0,2,0,2,0,8,2


In [26]:
# import k-means from clustering stage
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters = 5, random_state = 0).fit(sconehot)

print(                                                                                  )
print('>>>>>>>>>>>>>>>>>>>>>>>> ::::: k-MEANS CLUSTERING RUN ::::: ')


>>>>>>>>>>>>>>>>>>>>>>>> ::::: k-MEANS CLUSTERING RUN ::::: 


### 
Showing Centers of Each Cluster

In [27]:
means = pd.DataFrame(kmeans.cluster_centers_)
means.columns = sconehot.columns
means.index = ['G1','G2','G3','G4','G5']
means['Total Sum'] = means.sum(axis = 1)
means.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G3,2.000000,1.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000,1.000000,2.000000,22.000000,0.000000,29.000000
G1,2.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.666667,0.666667,1.666667,1.000000,12.333333,1.333333,19.666667
G5,0.666667,0.333333,0.0,3.333333e-01,0.000000e+00,3.333333e-01,1.333333,0.000000,1.000000,0.666667,6.666667,1.000000,12.333333
G4,0.000000,0.000000,0.0,0.000000e+00,1.000000e+00,0.000000e+00,1.000000,0.000000,4.000000,1.000000,3.000000,2.000000,12.000000
G2,0.333333,0.166667,0.5,1.387779e-17,1.387779e-17,1.387779e-17,0.333333,0.000000,0.666667,0.333333,2.166667,0.333333,4.833333


### Result 

__Best Group:__        G3

__Second Runner up:__  G1

__Third Runner up:__   G5
